In [198]:
from __future__ import print_function
import numpy as np #for mathematical operations
import matplotlib.pyplot as plt #for plotting output
import IPython.display #for audio output
import librosa #for audio 
import math
from collections import Counter
import librosa.display #for visualisation
%matplotlib inline 

#http://ibeat.org/piano-chords-free/
#http://biblio.telecom-paristech.fr/cgi-bin/download.cgi?id=10050
#https://hal.archives-ouvertes.fr/hal-00656352/document
#http://www.nyu.edu/classes/bello/MIR_files/tonality.pdf

In [259]:
#Load file for chord detection
audio_sample = '/home/student/data/cs4065/demo/Hardstyle.mp3'
audio_sample_y, audio_sample_sr = librosa.load(audio_sample)
y_harmonic, y_percussive = librosa.effects.hpss(audio_sample_y)

# Get the duration of the song (float)
duration = librosa.get_duration(y=audio_sample_y, sr=audio_sample_sr)

In [260]:
audio_sample_chroma = librosa.feature.chroma_cqt(y=y_harmonic, sr=audio_sample_sr)

# Make a new figure
#plt.figure(figsize=(12,4))

# Display the chromagram
#librosa.display.specshow(audio_sample_chroma, sr=audio_sample_sr, x_axis='time', y_axis='chroma', vmin=0, vmax=1)
#plt.title('Chromagram')
#plt.colorbar()

#plt.tight_layout()

In [273]:
# The amount of seconds each slice will be put
N = 10

# The Threshold on which notes will be counted
THRESHOLD = 0.9

# Transpose chroma so that it becomes [Timeslice][Notes]
chroma_transposed = np.transpose(audio_sample_chroma) 

# Number of samples per 'N' seconds
nsamples = int((len(chroma_transposed) / duration) * N)
print("Number of samples per slice {}".format(nsamples))

#number of periods of 'N' seconds
nperiods = int(math.ceil(len(chroma_transposed) / nsamples))
print("Number of periods per slice {}".format(nperiods))


pstart = 0 #start of the period 
pend = pstart + nsamples
periods = np.zeros((nperiods, nsamples, 12))
chroma = np.zeros((nperiods, 12))

for i in range(0, nperiods):
    chroma[i] = np.array([sum(a) / len(a) for a in zip(*chroma_transposed[pstart:pend])])
    for slice in chroma_transposed[pstart:pend]:
        for key in range(0, 12):
            chroma[i][key] += slice[key] if slice[key] > THRESHOLD else 0
    
    max = np.amax(chroma[i])
    for key in range(0, 12):
        chroma[i][key] /= max
        
    pstart = pend+1
    pend = pstart+nsamples

Number of samples per slice 430
Number of periods per slice 28


In [274]:
keys = [{'name': 'C-maj', 'scale': ['A', 'B', 'C', 'D', 'E', 'F', 'G']},
        
          {'name': 'G-maj', 'scale': ['A', 'B', 'C', 'D', 'E', 'F#/Gb', 'G']},
          {'name': 'F-maj', 'scale': ['A', 'A#/Bb', 'C', 'D', 'E', 'F', 'G']},
        
          {'name': 'D-maj', 'scale': ['A', 'B', 'C#/Db', 'D', 'E', 'F#/Gb', 'G']},
          {'name': 'Bb-maj', 'scale': ['A', 'A#/Bb', 'C', 'D', 'D#/Eb', 'F', 'G']},
        
          {'name': 'A-maj', 'scale': ['A', 'B', 'C#/Db', 'D', 'E', 'F#/Gb', 'G#/Ab']},
          {'name': 'Eb-maj', 'scale': ['G#/Ab', 'A#/Bb', 'C', 'D', 'D#/Eb', 'F', 'G']},
        
          {'name': 'E-maj', 'scale': ['E', 'F#/Gb', 'G#/Ab', 'A', 'B', 'C#/Db', 'D#/Eb']},
          {'name': 'Ab-maj', 'scale': ['G#/Ab', 'A#/Bb', 'C', 'C#/Db', 'D#/Eb', 'F', 'G']},
        
          {'name': 'B-maj', 'scale': ['B', 'C#/Db', 'D#/Eb', 'F', 'F#/Gb', 'G#/Ab', 'A#/Bb']},
          {'name': 'C#-maj', 'scale': ['C#/Db', 'D#/Eb', 'F', 'F#/Gb', 'G#/Ab', 'A#/Bb', 'C']},
        
          {'name': 'F#-maj', 'scale': ['F#/Gb', 'G#/Ab', 'A#/Bb', 'B', 'C#/Db', 'D#/Eb', 'F']}
         ]

In [275]:
def NoteToNumber(note):
    if note == 'C':
        return 0
    elif note == 'C#/Db':
        return 1
    elif note == 'D':
        return 2
    elif note == 'D#/Eb':
        return 3
    elif note == 'E':
        return 4
    elif note == 'F':
        return 5
    elif note == 'F#/Gb':
        return 6
    elif note == 'G':
        return 7
    elif note == 'G#/Ab':
        return 8
    elif note == 'A':
        return 9
    elif note == 'A#/Bb':
        return 10
    elif note == 'B':
        return 11

In [276]:
results = []
for slice in chroma:
    

    best_score = -100
    best_key = {'name': 'C-maj', 'scale': ['A', 'B', 'C', 'D', 'E', 'F', 'G']}
    
    for key in keys:
        score = 0
        for note in ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']:
            intensity = slice[NoteToNumber(note)]*10
            if note in key['scale']:
                score += intensity
            else:
                score -= intensity

        if score > best_score:
            best_score = score
            best_key = key
            
    print(best_key['name'])
    print(slice)
    results += {best_key['name']}

Counter(results).most_common()

Eb-maj
[ 0.20313124  0.05481764  0.05961667  0.07512515  0.15462865  0.29168593
  0.48969926  1.          0.64882144  0.22771744  0.21335903  0.29775271]
Ab-maj
[ 0.03698919  0.01022684  0.01006357  0.0326395   0.08470312  0.42101042
  0.43232608  1.          0.53265399  0.28367775  0.05805477  0.19703102]
Ab-maj
[ 0.08525631  0.0390205   0.02743507  0.0092284   0.01481121  0.0568181
  0.04171897  1.          0.24982496  0.03029671  0.01497149  0.09529265]
Ab-maj
[  2.83942957e-04   2.85927337e-04   2.85810175e-04   3.27218261e-04
   2.77028289e-03   1.22768920e-02   4.49740296e-02   1.00000000e+00
   2.04437209e-01   8.14800594e-04   5.32137982e-04   2.52807435e-04]
Eb-maj
[  2.96036277e-04   3.46368691e-04   2.79609285e-03   3.72153910e-04
   4.39967742e-04   1.45223713e-02   7.90261005e-02   1.00000000e+00
   2.36980756e-01   8.62904904e-04   5.42473749e-04   2.76940745e-04]
Eb-maj
[  2.28694666e-02   1.53468758e-02   2.75711012e-02   9.80802245e-03
   1.28680637e-02   1.30083325e-0

[('G-maj', 14), ('Eb-maj', 9), ('Ab-maj', 3), ('B-maj', 2)]